# 00: Notes on Go Programming Language Specification

This section is a summary of https://go.dev/ref/spec

## Objectives 

Understand at a high-level, Go Programming Language Spec, to be able to follow other articles.

## Introduction

Go is a general-purpose language designed with systems programming in mind. It is strongly typed and garbage-collected and has explicit support for concurrent programming.

In Go, programs are constructed from **packages**, whose properties allow efficient management of dependencies.

## Comments

Go allows for both line comments and general comments

```go
// line comment

/*
  General comment
*/
```

## Integer literals

An integer literal is a sequence of digits representing an integer constant.

An optional prefix sets a non-decimal base:

```go
binInt := 0b10 // binary
binInt = 0B10  // binary

octInt := 0o71 // octal
octInt = 0O71  // octal

hexInt := 0xFA  // hex
hexInt := 0xfa  // hex
hexInt = 0Xfa   // hex
hexInt = 0XFA   // hex
```

An underscore character can be used after a base prefix or between digits for readability:

```go
bigNum := 123_456_789
hexNum := 0x_FA
```

## Imaginary literals

Go allows you to work with complex numbers consisting on a real and an imaginary part.

```go
c := 1 + 2i // represents the complex number (1+2i)
c = -2.712i
```

## Rune literals

A rune literal represents a rune constant, an integer value identifying a Unicode code point.

A rune literal is expressed as one or more characters enclosed in single quotes, as in 'x' or '\n'.

## String literals

Two forms of string literals are supported: raw string literals and interpreted string literals.

Raw string literals are character sequences between *back quotes* as in \`foo\`. Any character may appear within the quotes, except for the back quote itself. Note that carriage return characters inside raw string literals will be discarded from the raw string value.

Interpreted string literals are character sequences between double quotes as in "bar". Any character may appear between the quotes except for newlines and unescaped double quotes. You can use the syntax "\unnn" to represent individual unicode points and "\xff" to represent individual bytes.

```go
s := "\u65e5\u672c\u8a9e"  // 日本語
```

## Types

### Slice types

A slice is a descriptor for a contiguous segment of an underlying array and provides access to a numbered sequence of elements from that array.

The length of a slice is never negative and can be identified by the function `len()`, and it can change during execution.

A slice, once initialized, is always associated with an underlying array that holds its elements. As a result, a slice shared storage with its array and with other slices of the same array; by contrast, distinct arrays always represent distinct storage.

The arrays underlying a slice may extend past the end of the slice. The *capacity* is a measure of that extent: it is the sum of the length of the slice and the length of the array beyond the slice. It can be discovered using the function `cap()`.

A slice of length up to that capacity can be created by slicing a new one from the original slice.

A new, initialized slide value for a given element type T may be created using the built-in funciton `make([]T, length, capacity)`, with `capacity` being optional. This function allocates a hidden array to which the returned slice refers.

Like arrays, slices are always one dimensional, but may be composed to construct higher-dimensional objects.

When using slices of slices, the inner lengths may vary dynamically. Moreover, the inner slices must be initialized individually.


### Struct types

A struct is a sequence of named elements called fields, each of which has a name and a type. Within a struct, non-blank fields must be unique.

```go
struct {}  // An empty struct

struc {
  x, y int
  u float32
  _ float32   // padding
  A *[]int
  F func()
}
```

A field declared with a type but no explicit field name is called an *embedded field*. An embedded field must be specified as a type name T or as a pointer to a non-interface type name *T, and T must not be a pointer type.

```go
// struct with embedded and regular fields
struct {
  T1        // field name T1
  *T2       // field name T2
  P.T3      // field name is T3
  *P.T4     // field name is T4
  x, y int  // regular fields
}
```

A field declaration might be followed by an optional literal tag which becomes an attribute for all the fields in the corresponding field declaration.

The tags are visible through a reflection interface and take part in type identify for structs, but are otherwise ignored:

```go
struct {
  name string "any tag is permitted"
}
```

### Pointer types

A pointer type denotes the set of all pointers to variables of a given type, called the *base type* of the pointer. The value of an initialized pointer is `nil`.

```go
*Point    // Pointer to a Point struct
*[4]int   // Pointer to an array of 4 ints
```

### Function types

A function type denotes the set of all functions with the same parameter and result types. The value of an uninitialized variable of function type is `nil`.

The final incoming parameter in a function signature may have a type prefixed with `...`. A function with such parameter is called *variadic* and may be invoked with zero or more arguments for that parameter.

```go
func()
func(x int) int
func(a, _ int, z float32) bool
func(a, b int, z float32) (bool)
func(prefix string, values ...int)
func(a, b int, z float64, opt ...interface{}) (success bool)
func(int, int, float64) (float64, *[int])
func(n int) func(p *T)
```

### Interface types

An interface defines a *type set*. A variable of interface type can store a value of any type that is in the type set of the interface. Such a type is said to *implement the interface*.

The value of an uninitialized variable of interface type is `nil`.

An interface element is either a method or a type element (a union of one or more type terms).

#### Basic interfaces

The most basic form of an interface specifies a possibly empty list of methods.

```go
// A simple File interface
interface {
  Read([]byte) (int, error)
  Write([]byte) (int, error)
  Close() error
}
```

More than one type can implement an interface, and a type can implement multiple interfaces.

Every type that is a member of the type set of an interface implements that interface.

All types implement the empty interface:

```go
interface{} // The empty interface
```

| NOTE: |
| :---- |
| The predeclared type `any` is an alias for the empty interface. |

The following interface specification declarare an interface called `Locker`:

type Locker interface {
  Lock()
  Unlock()
}

#### Embedded interface

An interface T may use a possibly qualified interface type name E as an interface element.

This is called *embedding* interface E in T.

```go
type Reader interface {
  Read(p []byte) (n int, err error)
  Close() error
}

type Writer interface {
  Write(p []byte) (n int, err error)
  Close() error
}

// Embedding Reader and Writer interfaces in ReadWriter
// ReadWriter's methods are Read, Write, and Close
type ReadWriter interface {
  Reader  // includes methods of Reader
  Writer  // includes methods of Writer
}

```

When embedding interface (as in the example above), methods with the same names must have identical signatures.

#### General interfaces

In their most general form, an interface element may also be an arbitrary type term T, or a term of the form ~T (specifying the underlying type T), or a union of terms $ t_1|t_2|...|t_n $.

By construction, an interface's type set never contains an interface type.

```go
// An interface representing the type int
interface {
  int
}

// An interface representing all types with underlying type int
interface {
  ~int
}

// An interface representing all types with underlying type int that implement the String method
interface {
  ~int
  String() string
}
```

In a term of the form ~T, the underlying type of T must be itself, and T cannot be an interface:

```go
type MyInt int

interface {
  ~int  // underlying type must be int
}
```

Also, you cannot use those types of interfaces in anything that is not a type constraint.

For example:

```go
// Error: cannot use type MyInt outside a type constraint
func MyFunc(n MyInt) {
	fmt.Println(n * 2)
}
```

| EXAMPLE: |
| :------- |
| See [01_interfaces-underlying-type](01_interfaces-underlying-type/) for a runnable example. |

Union elements denote unions of type sets:

```go
type Float interface {
  ~float32 | ~float64
}
```

As in the case above, you cannot use that itnerface outside of a type constraint.

As a result, its use will be limited to things like:

```go
type Float interface {
	~float32 | ~float64
}


func Double[T Float](n T) float64 {
	return float64(n) * 2
}
```

| EXAMPLE: |
| :------- |
| See [01_interfaces-underlying-type](01_interfaces-underlying-type/) for a runnable example. |

Interfaces that are not basic may only be used as type constraints, or as elements of other interfaces used as constraints.

```go
var x Float   // illegal

func double(x Float) float64 {  // illegal

}
```

#### Implementing an interface

A type T implements an interface I if:
+ T is not an interface and is an element of the type set of I; or
+ T is an interface and the type set of T is a subset of the type set of I.

### Map types

A map is an unordered group of elements of one type, called the element type, indexed by a set of unique keys of another type, called the key type.

The value of an uninitialized map is `nil`.

The comparison operators (`==` and `!=`) must be fully define for operands of the key type, and therefore, the key type must not be a function, map, or slice. If the key type is an interface type, these comparison operators must be defined for the dynamic key values.

The number of map elements is called its length, and it can be discovered using `len()`.

```go
map[string]int
map[*T]struct{ x, y float64 }
map[string]interface{}
```

A new, empty map value is made using the built-in function `make()`:

```go
make(map[string]int)
make(map[string]int, 100)  // optional capacity hint
```

### Channel types

A channel provides a mechanism for concurrently executing functions to communicate by sending and receiving values of a specified element type.

The optional `<-` operator specifies the channel direction. If a direction is given, the channel becomes directional, otherwise it is bidirectional.

```go
chan T          // channel to send and receive values of type T
chan<- float64  // channel to send float64s
<-chan int      // channel to receive ints
```

A new, initialized channel value can be created using the built-in function `make()`:

```go
make(chan int, 100)   // the capacity is optional, but when given makes the channel buffered
```

If the capacity is zero, the channel is unbuffered and communication succeeds only when both a sender and receiver are ready.

Otherwise, the channels is buffered and communication succeeds without blocking if the buffer is not full (sends) or not empty (receives). A nil channel is never ready for communication.

Channels act as first-infirst-out queues. For example, if one goroutine sends values on a channel and a second goroutine receives them, the values are received in the order sent.

A single channel may be used in send statement, receive operations, and calles to `cap()` and `len()` by any number of goroutines without further synchronization.

### Type identity

Two types are either identical or different.

A named type is always different from any other type. Otherwise, two types are identical if their underlying type literals are structurally equivalent.

### Assignability

A value `x` of type `V` is assignable to a variable of type `T` if othe of the following conditions apply:

+ V and T are identical
+ V and T have identical underlying types but are not type parameters and at least one of V or T is not a named type.
+ V and T are channel types with identical element types, V is a bidirectional channel, and at least one of V or T is not a named type.
+ T is an interface type, but not a type parameter, and x implements T.
+ `x` is the predeclared identifier `nil` and `T` is a pointer, function, slice, map, channel, or interface type, but not a type parameter.
+ `x` is an untyped constant representable by a value of type `T`.


## Exported identifiers

An identifier may be exported to permit access to it from another package.

An identifier is exported if both:
1. the first character of the identifier is Unicode uppercase letter; and
2. the identifier is declared in the package block or it is a field name or method name.

## Iota

Within a constant declaration, the predefined identifier iota represents successive untype integer constants.

Its value is the index of the respective ConstSpec in that constant declaration, starting at zero.

It can be used to construct a set of related constants (i.e., an enum):

```go
const (
  c0 = iota   // c0 == 0
  c1 = iota   // c1 == 1
  c2 = iota   // c2 == 2
)
```

You can also use the implicit repetition:

```go
const (
  Sunday = iota   // Sunday == 0
  Monday          // Monday == 1
  Tuesday         // ...
)
```

Things start getting interesting when you use iota as in the examples below:

```go
const (
  first float64 = iota	// 2^0
  second = 1 << iota		// 2^1
  third									// 2^2
  fourth								// 2^3
)
```

```go
const (
  _ float64 = iota
  KB = 1 << (10 * iota) // KB == (1 << 10) == 2^10
  MB										// MB == (1 << 20) == 2^20
  GB										// GB == (1 << 30) == 2^30
  TB										// TB == (1 << 40) == 2^40
)
```

## Type declarations

Type declarations binds an identifier (the type name) to a type. 

They come in two forms:
+ alias declarations: `type nodeList = []*Node`
+ type definitions: `type language string`

A defined type may have methods associated with it, but it does not inherit any methods bound to the given type.

```go
type Mutex struct { /* mutex fields */ }
func (m *Mutex) Lock() { /* ... */ }
func (m *Mutex) Unlock() { /* ... */ }

type NewMutex Mutex   // same composition but method set is empty
```


## Type parameter declarations

A type parameter list declares the type parameters of a generic function or type declaration. It is enclosed in square brackets rather than parentheses:

```go
[P any]
```

And in a function declaration it'll look like:

```go
func min[T ~int|~float64](x, y T) T {
  if x < y {
    return x
  } 
  return y
}
```

## Function declarations

A function declaration without type parameters may omit the body to provide the signature of a function implemented outside Go, such as an assembly routine:

```go
flushCache(begin, end uintptr)  // implemented externally
```

## Method declarations

A method is a function with a receiver.

If the receiver base type is a generic type, the receiver specification must declare corresponding type parameters for the method to use:

```go
type Pair[A, B any] struct {
  a A
  b B
}

func (p Pair[A, B]) Swap() Pair[B, A] { /* ... */ }
func (p Pair[First, _]) First() First { /* ... */ }
```

## Expressions

### Composite literals

Composite literals consist of the type of the literal followed by a brace-bound list of elements. Each element may optionally be preceded by a corresponding key.

```go
type Point3D struct { x, y, z float64 }
type Line struct { p, q Point3D }

origin := Point3D{}   // zero value for Point3D
line := Line{origin, Point3D{y: -4, z: 12.3}} // zero value for line.q.x
```

### Function literals

A function literal represents an anonymous function. Function literals cannot declare type parameters.

```go
// a function literal
func(a, b int, z float64) bool { return a * b < int(z) }
```

A function literatl can be assigned to a variable or invoked directly:

```go
f := func(x, y int) int { x + y }
func(a, b int, z float64) bool { return a * b < int(z) } (1, 2, 3.1415)
```

Function literals are closures: they may refer to variables defined in a surrounding function. Those variables are then shared between the surrounding function and the function literal, and they survive as long as they are accessible.

## Full slice expression

The primary expression:

```go
a[low : high : max]
```

constructs a slice of the same type, and with the same length and elements as the simple slice expression a[low:high]. Additionally, it controls the resulting slice's capacity by setting it to max -low.

Only the first index may be omitted; it defaults to 0.

```go
a := [5]int{1, 2, 3, 4, 5}
t := a[1:3:5]
```

make t a slice of type []int, length 2, capacity 4 and elements:

```go
t[0] == 2
t[1] == 3
```

## Type assertions

For an expression `x` of interface type, but not a type parameter, and a type `T`, the primary expression:

```go
x.(T)
```

asserts that `x` is not nil and that the value store in x is of type `T`.

The notation `x.(T)` is called a type assertion.

If `T` is not an interface type, `x.(T)` asserts that the dynamic type of `x` is identical to the type `T`.

When using this special form:
```go
v, ok := x.(T)
```

The value of `ok` is true if the assertion holds. Otherwise, it is `false` and the value `v` is the zero value for type `T` and no run-time panic occurs. If the boolean value is not present and the assertion does not hold, a run-time panic will occur.

## Function calss: passing arguments to ... parameters

If a function `f` is variadic with a final parameter `p` of type `...T`, then within `f` the type of `p` is equivalent to type `[]T`.

If `f` is invoked with no actual arguments for `p`, the value passed to `p` is `nil`. Otherwise, the value passed is a new slice of type `[]T` with a new underlying array whose successive elements are the actual arguments, which all must be assignable to `T`.

Given the following scenario:

```go
func Greeting(prefix string, who ...string) { ... }

s := []string{"Jason", "Idris"}
Greeting("Hello to", s...)
```

`s...` will be passed unchanged as the value for the `...string` parameter.

## Operators: Address operators

For an operand `x` of type `T`, the address operation `&x` generates a pointer of type `*T` to `x`.

## Operators: Receive operator

For an operand `ch` whose core type is channel, the value of the receive operation `<-` is the value received from the channel `ch`.

The channel direction must permit receive operations, and the type of the receive operaation is the element type of the channel.

The expression blocks until a value is available. Receiving from a `nil` channel blocks forever. A receive operation on a closed channel can always proceed immediately, yielding the element type's zero value.

```go
v1 := <- ch
```

A receive operation can use the special form:

```go
v, ok := <-ch
```

yields an additional untyped boolean result reporting whether the communication succeeded. The value of `ok` is true if the value received was delivered by a successful send operation to the channel, or false if it is a zero value generated because the channel is closed and empty.

## Conversions

A conversion changes the type of an expression to the type specified by the conversion. A conversion may appear literally in the source, or it may be implied by the context in which an expression appears.

```go
float32(2.718)

int(1.2)

*Point(p)     
*(Point(p)) // same as above

func()(x)
(func())(x) // same as above
```

## Send statements

A send statement sends a value on a channel. The channel expression's core type must be a channel, the channel direction must permit send operations, and the type of the value to be sent must be assignable to the channel's element type.

```go
ch <- 3   // send value 3 to channel ch
```

## IncDec statements

The `++` and `--` increment or decrement their operands by the untyped constant 1.

```go
x++ // same as x += 1
x-- // same as x -= 1
```

## Assignments: tuple assignments

A typle assignment assigns the individual elements of a multi-valued operation to a list of variables:

```go
func f() (int, int) { ... }
x, y := f()

one, two, three := '一', '二', '三'
```

The blank identifier `_` can be used to ignore right-hand side values in an assignment:

```go
x, _ = f()
```

## If statements

If statements specify the conditional execution of two branches according to the value of a boolean expression.

```go
if x > max {
  x = max
}
```

The expression may be preceded by a simple statement, which executed before the expression is evaluated:

```go
if x := f(); x < y {
  return x
} else if x > z {
  return z
} else {
  return y
}
```

## For statements

In its simplest form, a for statement specifies the repeated execution of a block as long as a boolean condition evaluates to true.

The condition is evaluated before each iteration. If the condition is absent, it is equivalent to the boolean value `true`:

```go
for a < b {
  a := 2
}
```

A for statement with a for clause is also controlled by its condition, but additionally it might specify an init and a post statement:

```go
// init statement; condition; post statement
for i := 0; i < 10; i++ {
  f(i)
}
```

Variables declared by the init statement are re-used in each iteration.

If non-empty, the init statement is executed once before evaluating the condition for the first iteration; the post statement is executed after each execution of the block.

## For statements with range clause

A for statement with a "range" clause iterates through all entries of an array, slice, string, or map, or values received on a channel.

| Range expression | 1st value | 2nd value |
| :--------------- | :-------- | :-------- |
| array or slice   | index     | value |
| string           | index     | rune  |
| map              | key       | value |
| channel          | element   | n/a   | 

## go statements

A "go" statement starts the execution of a function call as an independent concurrent thread of control, or goroutine, within the same address space.

```go
go Server()
go func(ch chan<- bool) { for { sleep(10); ch <- true }} (c)
```

## Select statements

A "select" statement chooses which of a set of possible send or receive operations will proceed. It looks similar to a "switch" statement but with the cases all referring to communication operations.

```go
select {
  case val1 <- x:
    // ... do something
  case val2 <- y:
    // ... do some other thing
}
```

And a more contrived example>

```go
var a []int
var c, c1, c2, c3, c4 chan int
var i1, i2 int

select {
case i1 = <-c1:
  print("received ", i1, " from c1\n")
case c2 <- i2:
  print("sent ", i2, " to c2\n")
case i3, ok := <-c3:  // same as i3, ok := (<-c3)
  if ok {
    print("received ", i3, " from c3\n")
  } else {
    print("c3 is closed\n")
  }
case a[f()] = <-c4:
  // same as:
  // case t := <-c4
  //   a[f()] = t
default:
  print("no communication\n")
}

// send random sequence of bits to c
for {
  select {
  case c <- 0:
  case c <- 1:
  }
}

// blocks forever
select {}
```

## Return statements

A return statement in a function `F` terminates the execution of `F`, and optionally provides one or more result values. Any functions deferred by F are executed before `F` returns to its caller.

In a function without a result type, a "return" statement must not specify any result values:

```go
func noResult() {
  return
}
```

The return value or values may be explicitly listed in the "return" statement. Each expression must be single-value and assignable to the corresponding element of the function's result type.

```go
func simpleF() int {
  return 2
}

func complexF() (re float64, im float64) {
  return 7.0, -4.0
}

func complexF2() (re float64, im float64) {
  return complexF()
}
```

The expression list may be empty if the function's result type specifies names for its result parameters:

```go
func complexF3() (re float64, im float64) {
  re = 7.0
  im = -4.0
  return
}

func Write(p []byte) (n int, _ error) {
  n = len(p)
  return
}
```

## Break statements

A "break" statement terminates the execution of the innermost "for", "switch", or "select" statement within the same function.

If there is a label, it must be that of an eclosing "for", "switch", or "select" statement, athat is the the one whose execution terminates:

```go
OuterLoop:
  for i = 0; i < n; i++ {
    for j = 0; j < m; j++ {
      switch a[i][j] {
        case nil:
          state = Error
          break OuterLoop
        case item:
          state = Found
          break OuterLoop
      }
    }
  }
```

## Continue statements

A "continue" statement begins the next iteration of the innermost enclosing "for" loop by advancing control to the end of the loop block. The "for" loop must be within the same function.

If there is a label, it must be that of an enclosing "for" statement, and that is the one whose execution advances:

```go
RowLoop:
  for y, row := range rows {
    for x, data := range row {
      if data == endOfRow {
        continue RowLoop
      }
      row[x] = data + bias(x, y)
    }
  }
```

## Goto statements

A "goto" statement transfers control tot he statement with the corresponding label within the same function.

## Fallthrough statements

A "fallthrough" statement transfers control to the first statement of the next case clause in an expression "switch" statement. It may be used only as the final non-empty statement in such a clause.

## Defer statement

A "defer" statement invokes a function whose execution is deferred to the moment the surrounding function returns, either because the surrounding function executed a "return" statement, reached the end of its function body, or because the corresponding gorouting is panicking.

The expression must be a function or method call; it cannot be parenthesized. Calls of built-in functions are restricted as for expression statements.

```go
// Prints Hello, world! in a contrived way
func main() {
  defer fmt.Println("world!")
  fmt.Println("Hello,")
}
```

## Built-in functions

Built-in functions are predeclared. They are called like any other function but some of them accept a type instead of an expression as the first argument.

They cannot be used as function values.

### Close

For an argument `ch` of type channel, the function `close()` records that no more values will be sent on the channel.

It is an error if `ch` is a receive only channel.

### Length and capacity

The built-in functions `len()` and `cap()` take arguments of various types and return a result of type `int`.

| Call | Argument type | Result |
| :--- | :------------ | :----- |
| len(s) | string type | string length in bytes |
|        | [n]T, *[n]T | array length (== n) |
|        | []T         | slice length |
|        | map[K]T     | number of defined keys in map |
|        | chan T      | number of elements queued in channel buffer |
|        | type parameter | (see below) |
| cap(s) | [n]T, *[n]T | array length (== n) |
|        | []T | slice capacity |
|        | chan T | channel buffer capacity |
|        | type parameter | (see below) |

If the argument is a type parameter `P`, the call `len(e)` or `cap(e)` must be valid for each type in P's type set.

### Allocation

The built-in function `new()` takes a type T, allocated storage for a variable of that type at run time, and returns a value of type `*T` pointing to it.

```go
new(T)

type S struct { a int; b float64 }
new(S)
```

### Making slices, maps and channels

The built-in function `make()` takes a type `T`, optionally followed by a type specific list of expressions.

The core type of `T` must be a slice, map, or channel. It returns a value of type `T` (not *T). The memory is initilized as described in the section on [initial values](#program-initialization-and-execution)

| Call | Core type | Result |
| :--- | :-------- | :----- |
| make(T, n) | slice | slice of type T with length n and capacity n |
| make(T, n, m) | slice | slice of type T with length n and capacity m |
| make(T) | map | map of type T |
| make(T, n) | map | map of type T with initial space for approximately n elements |
| make(T) | channel | unbuffered channel of type T |
| make(T, n) | channel | buffered channel of type T, buffer size n |

```go
s := make([]int, 10, 100)
c := make(chan int, 10)
m := make(map[string]int, 100)
```

### Appending to and copying slices

The built-in functions `append()` and `copy()` assist in common slice operations.

```go
append(s S, x ...E) S   // core type of S is []E
```

If the capacity of S is not large enough to fit the additional values, append allocates a new, sufficiently large underlying array that fits both the existing slice elements and the additional values. Otherwise, append re-uses the underlying array.


```go
s0 := []{0, 0}
s1 := append(s0, 2)               // append a single element
s2 := append(s1, 3, 5, 7)         // append multiple elements
s3 := append(s2, s0...)           // append a slice
s4 := append(s3[3:6], s3[2:]...)  // append overlapping slice

var b []byte
b = append(b, "bar"...) // append string contents 'b', 'a', 'r'
```

The function `copy()` copes slice elements from a source `src` to a destination `dst` and returns the number of elements copied.

```go
copy(dst, src []T) int
copy(dst []byte, src string) int
```

Examples:

```go
var a = [...]int{0, 1, 2, 3, 4, 5, 6, 7}
var s = make([]int, 6)
var b = make([]byte, 5)

n1 := copy(s, a[0:])  // n1 == 6,  s == []int{0, 1, 2, 3, 4, 5}
n2 := copy(s, s[2:])  // n2 == 4, s == []int{2, 3, 4, 5, 4, 5}
n3 := copy(b, "Hello, World!") // n3 == 5, b == []byte("Hello")
```

### Deletion of map elements

The built-in function `delete()` removes the element with key `k` from a map `m`.

```go
delete(m, k)  // remove element m[k] from map m
```

### Manipulating complex numbers

The following functions can be used to assemble and disassemble complex numbers.

```go
func complex(realPart, imaginaryPart floatT) complexT
func real(complexT) floatT
func imag(complexT) floatT
```

### Handling panics

Two built-in functions assist in reporting and handling run-time panics and program-defined error conditions.

```go
func panic(interface{})
func recover() interface{}
```

While executing a function `F`, an explicit call to `panic()` or a run-time panic terminates the execution of `F`. Any functions deferred by `F` are then executed as usual. Next, any deferred functions run by F's caller are run, and so on up to any deferred by the top-level function in the executing goroutine.

At that point, the program is terminated and the error condition is reported, including the value of the argument to ´panic()`. 

This termination sequence is called *panicking*.

```go
panic(42)
panic("unreachable")
panic(Error("cannot parse"))
```

The `recover()` function allows a program to manage behavior of a panicking goroutine.

The `protect()` function in the example below invokes the function argument `g` and protects callers from run-time panics raised by `g`.

```go
func protect(g func()) {
  defer func() {
    log.Println("done")
    if x := recover(); x != nil {
      log.Printf("run-time panic recovering: %v", x)
    }
  }()
  log.Println("execution start")
  g()
}
```

## Packages

Go programs are constructed by linking together packages.

A package in turn is constructed from one or more source files that together declare constants, types, variables, and functions belonging to the package and which are accessible in all files of the same package. Those elements may be exported and used in another package.

### Import declarations

An import declaration states that the source file containing the declaration depends on functionality of the imported package and enables access to the exported identifiers of that package.

The package name will be used in qualified identifiers to access exported identifiers of the package within the importing source file. If that package name is omitted, it defaults to the identifier in the package clause of the imported package.

When using a `.` as the package name, you will be allowed to use the exported identifiers unqualified.

```go
import "lib/math"   // usage: math.Sin()
import m "lib/math" // usage: m.Sin()
import . "lib/math" // usage: Sin()
```

## Program Initialization and Execution

### The zero value

When storage is allocated for a variable, and no explicit initialization is provided, the variable or value is given a default value.

Each element of such a variable or value is set to the zero value for its type:
+ `false` for booleans
+ `0` for numeric types
+ `""` for strings
+ `nil` for pointers, functions, interfaces, slices, channels, and maps.

The initialization is done recursively, so for each instance each element of an array of structs will have its fields zeroed if no value is specified.

### Package initialization

Within a package, package-level variable initialiatization proceeds stepwise, with each step selecting the variable ealiest in the declaration order which has no dependencies on uninitialized variables.

For example, given the declarations:

```go
var (
  a = c + b   // a == 5 + 4
  b = f()     // b == 4
  c = f()      // c == 5
  d = 3
)

func f() int {
  d++
  return
}
```

### Program execution

A complete program is created by linking a single, unimported package called the *main* package with all the packages it imports, transitively.

The main package must have package name `main` and declare a function `main` that takes no arguments and returns no value.

```go
func main() {
  ...
}
```

### Errors

The predeclared type error is defined as:

```go
type error interface {
  Error() string
}
```

It is the conventional interface for representing an error condition, with the nil value representing no error.

For instance, a function to read data from a file might be defined as:

```go
func Read(f *File, b []byte) (n int, err error)
```

### Run-time panics

Execution errors such as attempting to index an array out of bounds trigger a *run-time panic*. This is equivalent to a call of the built-in function `panic()` with a value of the implementation-defined interface type `runtime`. That type satisfies the predeclared interface type `error`.

## System considerations

### Package `unsafe`

The built-in package `unsafe`, accesible through `import "unsafe"` provides facilities for low-level programming including operations that violate the type system.